In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json
/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json


In [2]:
df = pd.read_json("/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json", lines =1)
for i in range(0, 26709):
    df.iloc[i,1] = df.iloc[i,1].replace ("'",'')
    
X = df.iloc[:,1]
y= df.iloc[:,2]

    

In [3]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

In [4]:
corpus = []
ps = PorterStemmer()
for i in range (0,26709):
    sent = re.sub("[^A-Za-z]", " ", X[i])
    sent = sent.lower().split()
    sent = [ps.stem(word) for word in sent if word not in set(stopwords.words("english"))]
    sent = " ".join(sent)
    corpus.append(sent)
    
    

In [5]:
cv = CountVectorizer(max_features = 1000)
X_processed = cv.fit_transform(X).toarray()

In [6]:
X_train,X_test,y_train,y_test =train_test_split(X_processed, y, test_size = 0.2, random_state  = 10, shuffle = True )

**Linear SVC classifier**

In [7]:
from sklearn.svm import LinearSVC
simple_classifier = LinearSVC()
simple_classifier.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [8]:
y_linearsvc_pred = simple_classifier.predict(X_test)
print(confusion_matrix (y_linearsvc_pred,y_test))
print(accuracy_score (y_linearsvc_pred,y_test))

[[2523  562]
 [ 508 1749]]
0.7997004867090978


In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [10]:
ann_classifier = Sequential()
ann_classifier.add(Dense(output_dim = 128, init ='uniform', activation = 'relu', input_dim = 1000 ))
ann_classifier.add(Dense(output_dim = 2, init ='uniform', activation = 'relu'))
ann_classifier.add(Dense(output_dim =1, init ='uniform', activation ='sigmoid'))

ann_classifier.compile(optimizer = 'adam', loss= 'binary_crossentropy', metrics = ['accuracy'])
ann_classifier.fit(X_train,y_train, batch_size = 100, nb_epoch= 10)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=1000, units=128, kernel_initializer="uniform")`
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=2, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


Epoch 1/10
21367/21367 [==============================] - 2s 88us/step - loss: 0.6359 - accuracy: 0.7139
Epoch 2/10
21367/21367 [==============================] - 1s 35us/step - loss: 0.5245 - accuracy: 0.8129
Epoch 3/10
21367/21367 [==============================] - 1s 34us/step - loss: 0.4835 - accuracy: 0.8226
Epoch 4/10
21367/21367 [==============================] - 1s 40us/step - loss: 0.4580 - accuracy: 0.8286
Epoch 5/10
21367/21367 [==============================] - 1s 35us/step - loss: 0.4403 - accuracy: 0.8327
Epoch 6/10
21367/21367 [==============================] - 1s 34us/step - loss: 0.4268 - accuracy: 0.8328
Epoch 7/10
21367/21367 [==============================] - 1s 34us/step - loss: 0.4158 - accuracy: 0.8345
Epoch 8/10
21367/21367 [==============================] - 1s 34us/step - loss: 0.4068 - accuracy: 0.8364
Epoch 9/10
21367/21367 [==============================] - 1s 35us/step - loss: 0.3991 - accuracy: 0.8370
Epoch 10/10
21367/21367 [==============================

In [11]:
ann_y_pred = ann_classifier.predict(X_test)
ann_y_pred = (ann_y_pred > 0.5)

print(confusion_matrix(ann_y_pred,y_test))
print(accuracy_score(ann_y_pred,y_test))

[[2562  577]
 [ 469 1734]]
0.804193186072632
